<a href="https://colab.research.google.com/github/Akira-13/CC0E5-PC2/blob/main/Informe_T%C3%A9cnico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Informe técnico

## Conceptos teóricos

### Función hash

Una función hash es cualquier función que mapea un conjunto de elementos de tamaño arbitrario a valores específicos. Este conjunto de valores específicos se denomina *tabla hash*. La propiedad más importante de una función hash es su **aleatoriedad**. Es decir, la probabilidad de que un elemento específico sea mapeado a un valor en la tabla hash es la misma para cualquier valor de la tabla.

\begin{align*}
  h:\mathcal{U}= \{0,1,...., u-1\} \rightarrow \{0,1,..., m-1\}\\
  \mathbb{P}(h(x) = t) = \frac{1}{m}
\end{align*}

### Universalidad

La universalidad es la propiedad de una función hash aleatoria (de una famila de funciones hash $\mathcal{H}$) en la que la probabilidad de que dos elementos colisionen (sean mapeados al mismo valor hash) en la tabla hash está en el orden de $\frac{1}{m}$.

\begin{align*}
  \mathbb{P}(h(x) = h(y)) = \mathcal{O}(\frac{1}{m}) \\
  \forall x \neq y \in \mathcal{U}
\end{align*}

### k-independencia

La k-independencia es la propiedad de una función hash aleatoria (de una famila de funciones hash $\mathcal{H}$) en la que, dado un conjunto de elementos, la probabilidad de que k elementos distintos sean mapeados a k valores hash distintos está en el orden de $\frac{1}{m^k}$. Esto significa que la función es independiente hasta k valores. La k-independencia implica universalidad.

\begin{align*}
  \mathbb{P}(h(x_1) = t_1\ \&\ h(x_2) = t_2\ \&\ ... \ \&\ h(x_k) = t_k) = \mathcal{O}(\frac{1}{m^k}) \\
  \text{Para }x_i\text{ diferentes}
\end{align*}

## Hashes por tabulación

### Hash por tabulación simple

El hash por tabulación simple es una de las funciones de hash más directas y con propiedades interesantes que la vuelve objeto de estudio.

En este hashing, el valor inicial (la clave) se entiende como un vector de claves de tamaño $c$, las cuales cada una tienen un tamaño de $r$ bits. Es decir, para la clave $x$ se tiene $x = (x_0, x_1, x_2,... x_c)$. Luego, se construyen $c$ tablas hash $h_i$, una para cada clave, con un total de $2^r$ valores aleatorios en cada una. Entonces, el hash $h(x)$ de una clave $x$ es:

\begin{align*}
h(x) = \bigoplus_{i \in [c]} h_i[x_i]
\end{align*}

Es decir, cada sub-clave se mapea a un valor aleatorio y cada uno de estos valores se combina mediante XOR para formar el hash final.

#### 3-independencia

Una de las propiedades más importantes de este tipo de hashing es su 3-independencia.

Asumiendo claves de dos bits que se dividen en sub-claves de 1 bit cada uno, se cumple que.
1. Para una sola clave $x_1$, se cumple la uniformidad de hashes (tiene la misma probabilidad de tomar cualquier valor hash).
2. Para dos claves $x_1$ y $y_1$ diferentes, hay al menos una posición en la que $x_i \neq y_i$, por lo que este valor distinto de $y_i$ puede tomar cualquier valor de la tabla hash y resultar en un hash aleatorio.
3. Para tres claves $x_1$, $y_1$ y $z_1$, el argumento se mantiene y existe un valor $x_i = y_i \neq z_i$ el cual hace que el hash de $z$ asuma cualquier valor.
4. Para cuatro valores esto ya no es posible, ya que cualquier clave compartirá un bit con alguna otra clave. Por lo tanto, después de aplicar aplicar el hash de tabulación, se tendrá que las claves siguen compartiendo un bit con otra clave y siempre se tendra:
\begin{align*}
h(x) \oplus h(w) \oplus h(y) \oplus h(z) = 0
\end{align*}
Lo cual no debería garantizarse en un hash 4-independiente.

### "Twisted" tabulation hashing

Esta método de hashing por tabulación actúa de forma similar a la tabulación simple. En primera instancia, se calcula el hash por tabulación de las sub-claves *excluyendo la primera*. Además, se calcula un valor *twister* como resultado de las operaciones XOR del valor de todas las sub-claves (NO de sus valores hash). Finalmente, el hash resultante será el XOR entre el hash por tabulación calculado inicialmente con el valor de la tabla hash de la primera subclave XOR el *twister*.

Expresado matemáticamente, siendo las tablas $h^*_i$ las tablas con los valores de las subclaves junto a valores aleatorios.

\begin{align*}
\begin{array}{rcl}
(t, h_{>0}) &=& \displaystyle\bigoplus_{i=1}^{c-1} h_i^\star[x_i]; \\
h(x) &=& h_{>0} \oplus h_0[x_0 \oplus t].
\end{array}
\end{align*}

En la implementación presentada en el repositorio, tomamos un enfoque más práctico hallando primero el hash por tabulación simple de la clave completa, hallando el valor *twister_index* como la combinación XOR de todas las sub-claves y mapeando esta clave a una tabla de valores aleatorios distinta a las tablas de las sub-claves, la cual retorna el *twister*. Este *twister* es finalmente combinado mediante XOR al hash por tabulación original para hallar el hash final.

\begin{align*}
\begin{array}{rcl}
h &=& \bigoplus_{i=0}^{c-1} T_i[x_i] \\
t &=& \bigoplus_{i=0}^{c-1} x_i \\
h_{\text{final}} &=& h \oplus T^*[t]
\end{array}
\end{align*}

Pese a las diferencias, la idea principal se mantiene: Se agrega un valor aleatorio final para aumentar la aleatoridad y la posible dependencia entre claves, acercándose a la 4-independencia para efectos prácticos.

### Hashing por tabulación doble

El hashing por tabulación doble es una forma directa y eficiente de obtener hashes de alta aleatoriedad y de alta independencia. Sencillamente, se aplica la función de hash por tabulación simple dos veces: Una vez para la clave original y otra para el resultado del primero. Esto asegura que cualquier patrón presente en las claves se disminuya drásticamente y en la práctica llega a estadísticas similares a la 4-independencia.

\begin{align*}
\begin{array}{rcl}
y &=& h_1(x) \\
h(x) &=& h_2(y)
\end{array}
\end{align*}

## Estructuras

Estos tipos de hashing se pueden utilizar en estructuras que requieren de hashing con uniformidad y aleatoriedad sobre seguridad criptográfica. Dos de estas estructuras son los filtros de Bloom y Cuckoo Hashing.

### Filtro de Bloom

Los filtros de bloom verifican la pertenencia de un elemento a un conjunto sin almacenarlo explícitamente, sino distintos hashes independientes del mismo elemento en *buckets* con una probabilidad controlable de falsos positivos. Para los hashes, se pueden usar distintos hashes por tabulación simple, asegurándonos de que las tablas generadas en cada uno sean distintas.

#### Falsos positivos

Los filtros de Bloom tienen la posibilidad de afirmar que un elemento pertenece a un conjunto como resultado de la colisión de funciones hash. Esta probabilidad se expresa de la siguiente forma:

\begin{align*}
P_{\text{fp}} = \left(1 - e^{-kn/m}\right)^{k}
\end{align*}

Siendo $k$ el número de funciones hash, $n$ el número de elementos a agregar y $m$ el tamaño del Bloom Filter.

### Cuckoo Hashing

## Análisis

### Uniformidad de hashes

Para analizar la uniformidad de los hashes, se realizaron tests chi-cuadrado de Pearson bajo la hipótesis nula de que la distribución de las funciones de hash por tabulación son uniformes:

1. Se calcularon los hashes de valores aleatorios y se mapearon a un arreglo de *buckets*, inicializados con contadores en $0$. Cuando el hash cae en uno de estos *buckets*, el contador en cada uno de estos aumenta en $1$.
2. Se comparan los contadores de estos buckets con el resultado teórico esperado: para una función hash perfectamente aleatoria, todos los buckets deben tener el mismo valor.
3. Se repite este proceso hasta obtener datos suficientes de estadísticos chi-cuadrados y su p-valor para el número de *buckets* dado (en las pruebas, 1000 *buckets*).

Como se ve de los resultados en el repositorio, los estadísticos rodean el valor promedio $999$ y los p-valores raramente bajan del valor frontera $0.05$ para todas las funciones de hash. Estos datos confirman el comportamiento aproximadamente uniforme de las funciones de tabulación, ya que las pruebas siguen en promedio una distribución chi-cuadrada con 999 grados de libertad (el valor promedio es igual a los grados de libertad en distribuciones chi-cuadrada) para grandes conjuntos de datos y sus p-valores en sus estadísticos mayormente no indican que se pueda descartar la hipótesis nula. Por lo tanto, se puede afirmar que los hashes por tabulación son altamente aleatorios y uniformes.

### Tasa de falsos positivos de Bloom Filter bajo hashes por tabulación

Como parte de los análisis, se comparó la tasa de falsos positivos teórica de un Bloom Filter con su tasa de falsos positivos real bajo el uso de hashes de tabulación simple. Los resultados demostraron que las tasas de falsos real se mantienen cercana a la tasa de falso positivos teórica. En otras palabras, las funciones de hash por tabulación simple tienen un comportamiento lo suficientemente aleatorio y uniforme para que la tasa de falsos positivos sea controlable y predecible en un filtro de Bloom.

### Rendimiento

#### Filtro de Bloom

El rendimiento del filtro de Bloom tiene un comportamiento interesante para tamaños pequeños de la estructura: este toma una cantidad considerablemente mayor (aunque todavía imperceptible) para verificar si un elemento pertenece al filtro para tamaños pequeños, entre 100 y 5000 aproximadamente. Para tamaños mayores, el tiempo de verificación se estabiliza. Una suposición de este comportamiento puede ser que, debido a que los filtros se llenan más rápidamente en filtros pequeños, la verificación de elementos tiene que revisar una mayor cantidad de bits antes de encontrar un bit nulo en comparación a un filtro más grande con más bits nulos. De igual modo, agregar elementos toma más tiempo, debido al trabajo de cambiar bits, y tiene un comportamiento irregular para tamaños pequeños.

En general, el comportamiento de los filtros de Bloom es el esperado en tamaños grandes: tiempo constante de inserción y verificación de elementos.

## Discusión

### Limitaciones

### Posibles extensiones

## Fuentes

- https://www.youtube.com/watch?v=R8-Fr6hgqxY&t=4821s
- https://arxiv.org/abs/1505.01523